# Week 6: Development of a cyclone through baroclinic instability
*MAQ - 32806, Chiel van Heerwaarden & Imme Benedict, 2017*

In this assignment you will investigate whether the large-scale flow is in geostrophic and thermal wind balance. For this, you will use ECMWF ERA-Interim data from the 1st of January 2016 3:00.
___

# Setting up Python and loading the data
We start the tutorial by loading the required Python packages and setting the figure properties.

In [ ]:
# Loading the packages.
import numpy as np              # Numpy is the fundamental package for scientific computing in Python.
import netCDF4 as nc            # NetCDF is the data format of the meteorological data that we use.
import matplotlib.pyplot as plt # Matplotlib is a scientific plotting package.
import datetime                 # Datetime is a package to deal with dates.
import cartopy.crs as ccrs
import cartopy
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from matplotlib.colors import LinearSegmentedColormap
from ipywidgets import interact

# The statement below enforces the plots to be put into this notebook, instead of in their own windows.
%matplotlib inline
plt.rcParams.update({'font.size': 11})          # Set the standard font size of the plots to 11pt.
plt.rcParams.update({'figure.figsize': [13,5]}) # Set the standard figure size.

In [ ]:
# Create custom color map similar to the NCAR NCL WhiteBlueGreenYellowRed
cdict = {'red':   ((   0/253., 255./255., 255./255.),
                   (  36/253., 157./255., 157./255.),
                   (  72/253.,  72./255.,  72./255.),
                   ( 108/253.,  73./255.,  73./255.),
                   ( 145/253., 250./255., 250./255.),
                   ( 181/253., 245./255., 245./255.),
                   ( 217/253., 211./255., 211./255.),
                   ( 253/253., 146./255., 146./255.)),
         'green': ((   0/253., 255./255., 255./255.),
                   (  36/253., 218./255., 218./255.),
                   (  72/253., 142./255., 142./255.),
                   ( 108/253., 181./255., 181./255.),
                   ( 145/253., 232./255., 232./255.),
                   ( 181/253., 106./255., 106./255.),
                   ( 217/253.,  31./255.,  31./255.),
                   ( 253/253.,  21./255.,  21./255.)),
         'blue':  ((   0/253., 255./255., 255./255.),
                   (  36/253., 247./255., 247./255.),
                   (  72/253., 202./255., 202./255.),
                   ( 108/253.,  70./255.,  70./255.),
                   ( 145/253.,  92./255.,  92./255.),
                   ( 181/253.,  45./255.,  45./255.),
                   ( 217/253.,  40./255.,  40./255.),
                   ( 253/253.,  25./255.,  25./255.))}

my_cmap = LinearSegmentedColormap('my_colormap', cdict,256)

___
Now, you load the data and read out the desired variables from a NetCDF file from the ECMWF ERA-Interim data archive. You are loading a file with data from 1 January 2016.

In [ ]:
# Loading the ERA data.
nc_file = nc.Dataset("baroclinic.nc", "r")
lat = nc_file.variables["latitude"][:]
lon = nc_file.variables["longitude"][:]
p = nc_file.variables["level"][:]*100.
time = nc.num2date(nc_file.variables["time"][:], nc_file.variables["time"].units)

# We load 3D fields of the two horizontal wind components, the geopotential and the temperature.
u     = nc_file.variables["u"][:,:,:,:]
v     = nc_file.variables["v"][:,:,:,:]
omega = nc_file.variables["w"][:,:,:,:]
Phi   = nc_file.variables["z"][:,:,:,:]
T     = nc_file.variables["t"][:,:,:,:]
msl   = nc_file.variables["msl"][:,:,:] # Mean sea level pressure.
cp    = nc_file.variables["cp"][:,:,:]  # Convective precipitation.
lsp   = nc_file.variables["lsp"][:,:,:] # Large-scale precipitation.

nc_file.close() # The file access is closed as no further data is needed now.

___
# Plotting the temperature and geopotential on a map
As a starting point you find below the temperature and geopotential at 500 hPa.

In [ ]:
n = abs(p-50000).argmin()

my_projection = ccrs.PlateCarree(central_longitude=0)

def plot_map(nt=0):
    fig1 = plt.figure()
    ax1 = plt.subplot(111, projection=my_projection)

    #ax1.add_feature(cartopy.feature.COASTLINE, linewidth=0.8)
    ax1.coastlines('50m', linewidth=0.8)
    ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)

    ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
    ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax1.xaxis.set_major_formatter(lon_formatter)
    ax1.yaxis.set_major_formatter(lat_formatter)

    Tmin, Tmax = T[:,n,:,:].min(), T[:,n,:,:].max()
    Phi_range = np.arange(0, 90000, 1000)
    cb = ax1.pcolormesh(lon, lat, T[nt,n,:,:], cmap=my_cmap) # We plot a colormesh using the gist_ncar colormap.
    fig1.colorbar(cb) # We add a colorbar to show the values of temperature.
    cz = ax1.contour(lon, lat, Phi[nt,n,:,:], Phi_range, colors='w', linewidths=1.5) # We plot the geopotential in contours.
    ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
    #cs = ax1.contour(lon, lat, msl[nt,:,:], 15, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.


    ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
    fig1.tight_layout() # With this function we make the figure fit as good as possible.
    
interact(plot_map, nt=(0, u.shape[0]));

___
Work out the assignment below. If you feel more comfortable in using Word, you can also work the assignment out in there. You can save the pictures by clicking right on them. You can use the Notebook from last week to look up how to calculate gradients.
___

In [ ]:
p_min = msl.min(axis=-1).min(axis=-1)

fig = plt.figure()
ax1 = plt.subplot(111)
ax1.plot(time, p_min, 'o-')
fig.tight_layout()

Study the system at the early development: 4 January 2017, 00:00:00 UTC. Is the system developing at its logical place?

In [ ]:
nt = 4
n = abs(p-30000).argmin()

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

Tmin, Tmax = T[:,n,:,:].min(), T[:,n,:,:].max()
Phi_range = np.arange(0, 110000, 2000)
msl_range = np.arange(90000, 105000, 1000)
#cb = ax1.pcolormesh(lon, lat, T[nt,n,:,:], cmap=plt.cm.viridis) # We plot a colormesh using the gist_ncar colormap.
#fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='b', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.

ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() # With this function we make the figure fit as good as possible.

In [ ]:
Omega = 7.292e-5
a_earth = 6.37e6
g = 9.81
R = 287.

In [ ]:
latrad = np.deg2rad(lat)
lonrad = np.deg2rad(lon)

# We initialize an array with the cosine of the latitude.
cos_lat = np.cos(latrad)

# Two arrays of zeros are initialized with the same shape as that of Z.
dPhidx = np.zeros(Phi.shape)
dPhidy = np.zeros(Phi.shape)

dPhidx[:,:,:,:] = (a_earth*cos_lat[np.newaxis,np.newaxis,:,np.newaxis])**(-1) \
                * np.gradient(u, axis=3) / np.gradient(lonrad[np.newaxis, np.newaxis, np.newaxis, :], axis=3)
dPhidx[:,:,:,:] = (a_earth)**(-1) \
                * np.gradient(v, axis=2) / np.gradient(latrad[np.newaxis,np.newaxis,:,np.newaxis], axis=2)

dudx = (a_earth*cos_lat[np.newaxis,np.newaxis,:,np.newaxis])**(-1) \
     * np.gradient(u, axis=3) / np.gradient(lonrad[np.newaxis, np.newaxis, np.newaxis, :], axis=3)
dvdy = (a_earth)**(-1) \
     * np.gradient(v, axis=2) / np.gradient(latrad[np.newaxis,np.newaxis,:,np.newaxis], axis=2)
div = dudx + dvdy
    
f = 2.*Omega*np.sin(latrad)
ug = -(1./f[np.newaxis,np.newaxis,:,np.newaxis])*dPhidy
vg =  (1./f[np.newaxis,np.newaxis,:,np.newaxis])*dPhidx
Vg = (ug**2 + vg**2)**.5
V = (u**2 + v**2)**.5

In [ ]:
lon_min = -90
lon_max = -10
lat_min = 30
lat_max = 65

In [ ]:
# Divergence plot.
nt = 4
n = abs(p-30000).argmin()

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

div_min, div_max = -2e-4, 2e-4
Phi_range = np.arange(0, 110000, 2000)
msl_range = np.arange(90000, 105000, 1000)
cb = ax1.pcolormesh(lon, lat, div[nt,n,:,:], vmin=div_min, vmax=div_max, cmap=plt.cm.seismic)
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.

ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() # With this function we make the figure fit as good as possible.

In [ ]:
dTdx = (a_earth*cos_lat[np.newaxis,np.newaxis,:,np.newaxis])**(-1) \
     * np.gradient(T, axis=3) / np.gradient(lonrad[np.newaxis, np.newaxis, np.newaxis, :], axis=3)
dTdy = (a_earth)**(-1) \
     * np.gradient(T, axis=2) / np.gradient(latrad[np.newaxis,np.newaxis,:,np.newaxis], axis=2)
Tgrad = (dTdx**2 + dTdy**2)**.5

In [ ]:
# Omega plot.
nt = 6
n = abs(p-60000).argmin()
n300 = abs(p-30000).argmin()

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

Phi_range = np.arange(0, 110000, 2000)
msl_range = np.arange(90000, 105000, 1000)
cb = ax1.pcolormesh(lon, lat, omega[nt,n,:,:], cmap=plt.cm.seismic, vmin=-4, vmax=4)
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n300,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
ax1.set_xlim(lon_min, lon_max)
ax1.set_ylim(lat_min, lat_max)
ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() # With this function we make the figure fit as good as possible.

In [ ]:
# Fronts plot.
nt = 6
n = abs(p-85000).argmin()
n300 = abs(p-30000).argmin()

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

Phi_range = np.arange(0, 110000, 2000)
msl_range = np.arange(90000, 105000, 1000)
cb = ax1.pcolormesh(lon, lat, Tgrad[nt,n,:,:], cmap=my_cmap, vmin=0, vmax=1.6e-4)
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n300,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
ax1.set_xlim(lon_min, lon_max)
ax1.set_ylim(lat_min, lat_max)
ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() # With this function we make the figure fit as good as possible.

In [ ]:
# Precipitation plots.
nt = 6
n = abs(p-85000).argmin()
n300 = abs(p-30000).argmin()

# Convective precipitation.
fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

Phi_range = np.arange(0, 110000, 2000)
msl_range = np.arange(90000, 105000, 1000)
cb = ax1.pcolormesh(lon, lat, cp[nt,:,:]*1e3, cmap=my_cmap, vmin=0, vmax=4)
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n300,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
#ax1.set_xlim(lon_min, lon_max)
#ax1.set_ylim(lat_min, lat_max)
ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() # With this function we make the figure fit as good as possible.

# Large-scale precipitation.
fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

Phi_range = np.arange(0, 110000, 2000)
msl_range = np.arange(90000, 105000, 1000)
cb = ax1.pcolormesh(lon, lat, lsp[nt,:,:]*1e3, cmap=my_cmap, vmin=0, vmax=4)
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n300,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
#ax1.set_xlim(lon_min, lon_max)
#ax1.set_ylim(lat_min, lat_max)
ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() # With this function we make the figure fit as good as possible.

# Total precipitation.
pr = cp + lsp

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

Phi_range = np.arange(0, 110000, 2000)
msl_range = np.arange(90000, 105000, 1000)
cb = ax1.pcolormesh(lon, lat, pr[nt,:,:]*1e3, cmap=my_cmap, vmin=0, vmax=4)
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n300,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
#ax1.set_xlim(lon_min, lon_max)
#ax1.set_ylim(lat_min, lat_max)
ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() # With this function we make the figure fit as good as possible.